In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

In [33]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
columns_name = ['target','id','date','flag','user','text']
df1 = pd.read_csv(r"C:\Users\DELL\Downloads\datasets\twitter _data.csv",names = columns_name, encoding='latin1')
df2 = df1.copy()

In [32]:
df2.head(5)

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [5]:
df2['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [6]:
# Convert 4 to 1
df2.replace({'target':{4:1}},inplace = True)

In [7]:
porter_stem = PorterStemmer()

In [8]:
def Stemming(content):
    stemmed_cont = re.sub('[^a-zA-Z]',' ',content)
    stemmed_cont = stemmed_cont.lower()
    stemmed_cont = stemmed_cont.split()
    stemmed_cont = [porter_stem.stem(word) for word in stemmed_cont if not word in stopwords.words('english')]
    stemmed_cont = ' '.join(stemmed_cont)
    return stemmed_cont

In [9]:
df2['stemmed_content'] = df2['text'].apply(Stemming)
df2.to_csv('stemmed_twitter data.csv',index = False)

In [10]:
twitter_data = pd.read_csv(r'stemmed_twitter data.csv')
twitter_data.dropna(inplace = True)

In [34]:
df2.head(5)

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [12]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [13]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [14]:
Y

array([0, 0, 0, ..., 1, 1, 1])

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

In [16]:
print(X.shape,X_train.shape, X_test.shape)

(1599505,) (1279604,) (319901,)


In [17]:
vectorizer = TfidfVectorizer()
# Converting the textual data to numeric data
vectorizer = TfidfVectorizer()

In [18]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [19]:
model =  LogisticRegression(max_iter = 1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [20]:
X_train_pred = model.predict(X_train)
train_model_accuracy = accuracy_score(Y_train, X_train_pred)
print("Accuracy Score on the training data:", train_model_accuracy)

Accuracy Score on the training data: 0.8024209052175517


In [21]:
X_test_pred = model.predict(X_test)
test_model_accuracy = accuracy_score(Y_test, X_test_pred)
print("Accuracy Score on the test data:", test_model_accuracy)

Accuracy Score on the test data: 0.7783220433821714


In [22]:
# Saving the trained model
filename = "sentiment_trained_model.joblib"
joblib.dump(model,filename)

['sentiment_trained_model.joblib']

In [26]:
# After training your model, save the vectorizer along with the model
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")

['tfidf_vectorizer.joblib']

In [27]:
 #Loading the trained model and vectorizer
model = joblib.load("sentiment_trained_model.joblib")
vectorizer = joblib.load("tfidf_vectorizer.joblib")

In [28]:
# Process the tweet
tweet = 'This is an awesome game for entertainment'
p_tweet = Stemming(tweet)

In [29]:
# Use the SAME vectorizer used for training to transform the tweet
tweet_tfidf = vectorizer.transform([p_tweet])  # Transform the tweet using the trained vectorizer

In [30]:
# Predict the sentiment
prediction = model.predict(tweet_tfidf)

In [31]:
# Print the result
if prediction[0] == 1:
    print("Positive")
else:
    print("Negative")

Positive
